In [37]:
from Bio import SeqIO
import sklearn.metrics as skmetrics
from matplotlib import pyplot
from sklearn.metrics import roc_auc_score
from sklearn import metrics
import pandas as pd
import numpy as np
import numpy as np


class FishGenes(object):
    def __init__(self, fish_id=None, fish_type=None, sequence=None,
                 fishes={'fishname': [], 'number': [], 'replace': []}):
        self.fish_id = fish_id
        self.fish_type = fish_type
        self.sequence = sequence
        self.fishes = {'fishname': [], 'number': [], 'replace': []}


def parse_fasta(reffilename, fish_type):
    Fish_List = []
    for rec in SeqIO.parse(reffilename, "fasta"):
        name, sequence = rec.id, str(rec.seq)
        Fish_List.append(FishGenes(fish_id=name, fish_type=fish_type, sequence=sequence, fishes={}))
    return Fish_List


def convert_files(reffilename, reffilename2):
    Fish_DeepWater = []
    Fish_ShalloWater = []

    Fish_DeepWater = parse_fasta(reffilename, fish_type='deepwater')
    Fish_ShalloWater = parse_fasta(reffilename2, fish_type='shallowater')
    return Fish_DeepWater, Fish_ShalloWater


def get_numbers(sequence, replace_sequence):
    repeated_elements = []
    repeated_idx = []
    repeated_sequence = []

    for idx, val in enumerate(sequence):
        if sequence.count(val) > 1:
            repeated_elements.append(val)
            repeated_idx.append(idx)
            repeated_sequence.append(replace_sequence[idx])
    return repeated_elements, repeated_sequence


def sequence_dictionary(repeat_sequence, replace_sequence):
    sequence_dictionary = dict()
    for i in range(len(repeat_sequence)):
        key = repeat_sequence[i]
        if key not in sequence_dictionary:
            sequence_dictionary[key] = []
        sequence_dictionary[repeat_sequence[i]].append(replace_sequence[i])
    return sequence_dictionary


def convert_panda(data,len_shallow_fish):
    for i in range(len(data)):
        key = list(data)[i]
#         print('len(data[key])',len(data[key]))
        while len(data[key]) < len_shallow_fish:
            data[key].append(None)
    df = pd.DataFrame.from_dict(data)
    return df


reffilename = '/Users/alena_paliakova/Google Drive/!Bioinf_drive/00_FishPr/02_Fishgenes_work/data/deepwater_al_tr.fasta'
reffilename2 = '/Users/alena_paliakova/Google Drive/!Bioinf_drive/00_FishPr/02_Fishgenes_work/data/shallowwater_al_tr.fasta'
df_combine=pd.DataFrame()
Fish_DeepWater, Fish_ShalloWater = convert_files(reffilename, reffilename2)


for i in range((len(Fish_DeepWater))):
    for j in range(len(Fish_ShalloWater)):
        Fish_DeepWater[i].fishes['fishname'].append(Fish_ShalloWater[j].fish_id)
        for k in range(len(Fish_ShalloWater[j].sequence)):

            if (Fish_DeepWater[i].sequence[k] != '-') and (Fish_ShalloWater[j].sequence[k] != '-') and (
                    Fish_DeepWater[i].sequence[k] != Fish_ShalloWater[j].sequence[k]):
                Fish_DeepWater[i].fishes['number'].append(k)
                Fish_DeepWater[i].fishes['replace'].append(
                    Fish_DeepWater[i].sequence[k] + Fish_ShalloWater[j].sequence[k])


for i in range(len(Fish_DeepWater)):
    sequence = Fish_DeepWater[i].fishes['number']
    replace_sequence = Fish_DeepWater[i].fishes['replace']
    sequence_data = sequence_dictionary(sequence, replace_sequence)
    df = convert_panda(sequence_data,len(Fish_ShalloWater))
    fishname = []
    for element in list(Fish_DeepWater[i].fishes['fishname']):
        fishname.append(element)

    fish_id = []
    for k in range(len(Fish_DeepWater[i].fishes['fishname'])):
        fish_id.append(Fish_DeepWater[i].fish_id)

    
    df.insert(0, 'deepwater', fish_id)

    df.insert(1, "shallowater", fishname)

    df_combine = df_combine.append(df)
    
    
df_deep=df_combine['deepwater']
df_shallow=df_combine['shallowater']
df_combine=df_combine.drop(['deepwater', 'shallowater'], axis=1)

df_combine=df_combine.reindex_axis(sorted(df_combine.columns), axis=1)

df_combine.insert(0, 'deepwater', df_deep)
df_combine.insert(1, 'shallowater', df_shallow)

df_combine.to_csv('/Users/alena_paliakova/Google Drive/!Bioinf_drive/00_FishPr/02_Fishgenes_work/Fishgenes.csv',
                        index=False)
# print(df_combine.to_csv('/Users/alena_paliakova/Google Drive/!Bioinf_drive/00_FishPr/02_Fishgenes_work/Fishgenes.csv',
#                         index=False))
# print(df_combine)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:113: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
